<a href="https://colab.research.google.com/github/snow-The/GW190521/blob/main/homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 依賴安裝
!pip install "ml4gw>=0.7.10" "gwpy>=3.0" "h5py>=3.12" "torchmetrics>=1.6" "lightning>=2.4.0" "rich>=10.2.2,<14.0"

In [ ]:
# === Cell 2: 數據下載與白化處理 ===
import torch
import numpy as np
from gwpy.timeseries import TimeSeries
from ml4gw.transforms import Whiten, SpectralDensity

# 1. 初始化設定
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 2. 設定參數 (GW190521 v4)
trigger_time = 1242442967.4
sample_rate = 2048
# 分析視窗: 事件前6秒到後2秒
start_time = trigger_time - 6
end_time = trigger_time + 2
# PSD 視窗: 用更長的背景數據來估算雜訊
psd_start = start_time - 64
psd_end = start_time
ifos = ["H1", "L1"]

# 3. 定義下載函數 (含 NaN 自動修復)
def get_data(detectors, start, end):
    tensors = []
    print(f"Downloading data for {start} to {end}...")
    for det in detectors:
        # 自動從 GWOSC 下載
        ts = TimeSeries.fetch_open_data(det, start, end, verbose=False)
        ts = ts.resample(sample_rate)
        # 檢查並填補 NaN
        if np.isnan(ts.value).any():
            ts.value = np.nan_to_num(ts.value)
        tensors.append(torch.from_numpy(ts.value.copy()).float())
    return torch.stack(tensors).to(device)

# 4. 執行流程
try:
    # 下載數據
    data_tensor = get_data(ifos, start_time, end_time).unsqueeze(0) # (1, 2, N)
    psd_tensor = get_data(ifos, psd_start, psd_end)

    # 定義白化模組
    spectral_density = SpectralDensity(sample_rate=sample_rate, fftlength=2, average="median").to(device)
    whiten = Whiten(fduration=2, sample_rate=sample_rate, highpass=20).to(device)

    # 計算 PSD 並白化
    psd = spectral_density(psd_tensor.double())
    whitened_data = whiten(data_tensor, psd).float()

    # 計算白化後的精確時長 (因為頭尾被切掉了)
    current_samples = whitened_data.shape[-1]
    precise_duration = current_samples / sample_rate

    print("✅ 數據處理完成！")
    print(f"Whitened Data Shape: {whitened_data.shape}")
    print(f"Valid Duration: {precise_duration} seconds")

except Exception as e:
    print(f"❌ Error: {e}")

In [ ]:
# === Cell 3: 頻譜圖、牛頓力學與 Ringdown 分析 ===
import matplotlib.pyplot as plt
from scipy.signal import spectrogram
from scipy.optimize import curve_fit

# --- A. 準備繪圖數據 ---
# 從 GPU 取回 L1 探測器的數據
strain_l1 = whitened_data[0, 1].cpu().numpy()

# 重建時間軸 (Whitening 切掉了頭尾各 1 秒)
plot_start = start_time + 1
plot_end = end_time - 1
time_axis = np.linspace(plot_start, plot_end, len(strain_l1))

# --- B. 繪製頻譜圖 (Spectrogram) ---
fs = sample_rate
nperseg = int(fs / 16) # 小窗口以獲得高時間解析度
noverlap = int(nperseg * 0.95)
f_vec, t_vec, Sxx = spectrogram(strain_l1, fs, nperseg=nperseg, noverlap=noverlap)
t_vec += plot_start # 修正時間偏移

plt.figure(figsize=(14, 10))

# 子圖 1: 頻譜圖與牛頓力學擬合
plt.subplot(2, 1, 1)
plt.pcolormesh(t_vec, f_vec, Sxx, shading='gouraud', cmap='viridis', vmin=0, vmax=15)

# 牛頓力學擬合 (0PN Inspiral)
G, c, M_solar = 6.674e-11, 3e8, 1.989e30
m1, m2 = 85 * M_solar, 66 * M_solar
chirp_mass = (m1 * m2)**(3/5) / (m1 + m2)**(1/5)

def newtonian_frequency(t, t_c):
    tau = t_c - t
    with np.errstate(invalid='ignore'):
        val = (c**3) / (8 * np.pi * G * chirp_mass) * \
              ((5 * G * chirp_mass) / (c**3 * tau))**(3/8)
    return val

t_model = np.linspace(trigger_time - 1, trigger_time, 500)
f_model = newtonian_frequency(t_model, trigger_time + 0.02)
plt.plot(t_model, f_model, 'r--', linewidth=2.5, label='Newtonian Prediction (Inspiral)')

plt.yscale('log')
plt.ylim(20, 150)
plt.xlim(trigger_time - 0.5, trigger_time + 0.2)
plt.ylabel("Frequency (Hz)")
plt.title(rf"GW190521 Analysis: Spectrogram vs Newtonian Fit ($M_{{total}} \approx 150 M_\odot$)")
plt.legend(loc='upper left')
plt.colorbar(label="Energy")


# --- C. Ringdown 擬合 (進階項目) ---
# 子圖 2: 時域波形與 Ringdown 擬合
plt.subplot(2, 1, 2)

# 1. 尋找峰值
search_mask = (time_axis > trigger_time - 0.1) & (time_axis < trigger_time + 0.1)
peak_loc_idx = np.argmax(np.abs(strain_l1[search_mask]))
# 還原到全域索引
global_indices = np.where(search_mask)[0]
peak_idx = global_indices[peak_loc_idx]
t_peak = time_axis[peak_idx]

# 2. 定義 Ringdown 區域 (峰值後 3ms 到 50ms)
start_fit = peak_idx + int(0.003 * fs)
end_fit = peak_idx + int(0.05 * fs)
t_ring = time_axis[start_fit:end_fit]
h_ring = strain_l1[start_fit:end_fit]

# 3. 定義阻尼正弦波模型
def ringdown_model(t, A, tau, f, phi):
    dt = t - t[0]
    return A * np.exp(-dt/tau) * np.cos(2*np.pi*f*dt + phi)

# 4. 執行擬合
try:
    # 初始猜測: A=max, tau=10ms, f=65Hz (IMBH特徵), phi=0
    p0 = [np.max(np.abs(h_ring)), 0.01, 65, 0]
    popt, _ = curve_fit(ringdown_model, t_ring, h_ring, p0=p0, maxfev=5000)

    # 繪圖
    plt.plot(time_axis, strain_l1, 'k-', alpha=0.3, label='Whitened Data')
    plt.plot(t_ring, ringdown_model(t_ring, *popt), 'r-', linewidth=2, label=f'Ringdown Fit (f={popt[2]:.1f}Hz)')

    plt.xlim(trigger_time - 0.05, trigger_time + 0.1) # 放大看合併區
    plt.xlabel("GPS Time (s)")
    plt.ylabel("Strain (Whitened)")
    plt.title(f"Ringdown Analysis: Frequency = {popt[2]:.2f} Hz, Damping Time = {popt[1]*1000:.2f} ms")
    plt.legend()

    print(f"✅ Ringdown Analysis Success!")
    print(f"   Estimated Frequency: {popt[2]:.2f} Hz")
    print(f"   Estimated Damping Time: {popt[1]*1000:.2f} ms")

except Exception as e:
    print(f"❌ Ringdown Fit Failed: {e}")
    plt.plot(time_axis, strain_l1, 'k-', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# === Cell 4: 進階項目 2 - 相對論擬合 (Relativistic Fit) ===
# 1. 安裝 PyCBC (LIGO 的標準分析庫)
# 這可能需要 1-2 分鐘
!pip install pycbc lalsuite

In [ ]:
from pycbc.waveform import get_td_waveform
import matplotlib.pyplot as plt
import numpy as np

# 2. 生成廣義相對論波形 (General Relativity Waveform)
print("Generating Relativistic Waveform (IMRPhenomXPHM)...")

# 參數設定 (參考 GW190521 v4)
m1_det = 95 # 探測器參考系質量
m2_det = 69
s1z = 0.7
s2z = 0.7

hp, hc = get_td_waveform(approximant="IMRPhenomXPHM",
                         mass1=m1_det,
                         mass2=m2_det,
                         spin1z=s1z,
                         spin2z=s2z,
                         delta_t=1.0/2048,
                         f_lower=20)

# 3. 將波形對齊到我們的數據 (關鍵修正)
hp.resize(len(hp)) # 確保長度正確

# 計算位移量：我們想把波形的「尾巴」對齊到 trigger_time
# 注意：不能直接改 sample_times，要改 start_time
current_end_time = hp.sample_times[-1]
time_shift = trigger_time - current_end_time

# === 修正點 ===
hp.start_time += time_shift
# =============

# 轉成 Numpy 方便繪圖
rel_time = hp.sample_times.numpy()
rel_strain = hp.numpy()

# 4. 繪圖：牛頓力學 vs 廣義相對論
plt.figure(figsize=(12, 8))

# A. 畫出原本的頻譜圖 (背景)
# 確保我們有上一段生成的變數
try:
    plt.pcolormesh(t_vec, f_vec, Sxx, shading='gouraud', cmap='viridis', vmin=0, vmax=15)
except NameError:
    print("⚠️ 警告: 找不到頻譜圖數據 (t_vec, Sxx)，請確保上一段代碼已執行。")

# B. 畫出相對論波形 (時域疊加)
# 視覺化技巧：放大並平移波形以便觀察形狀
scale_factor = 300 # 放大倍率
offset_freq = 60   # 平移到 60Hz 處

plt.plot(rel_time, rel_strain * scale_factor * 1e19 + offset_freq,
         'w-', linewidth=1.5, alpha=0.9, label='Relativistic Waveform (IMRPhenom)')

# C. 標示合併時刻
plt.axvline(trigger_time, color='w', linestyle=':', alpha=0.5)

# 設定範圍
plt.xlim(trigger_time - 0.5, trigger_time + 0.2)
plt.ylim(20, 150)
plt.xlabel("GPS Time (s)")
plt.ylabel("Frequency (Hz)")
plt.title(f"Advanced Analysis: General Relativity Fit (IMRPhenomXPHM)\nOverlaying Waveform on Spectrogram")
plt.legend(loc='upper left')
plt.colorbar(label="Spectral Energy")

plt.show()

print("✅ 相對論擬合完成！")

## 進階項目 3: 基於深度卷積神經網路 (CNN) 的重力波訊號辨識

### 1. 研究動機與挑戰
GW190521 是一個極短持續時間（< 0.1秒）的中等質量黑洞合併事件。由於其信號特徵主要集中在合併 (Merger) 與鈴振 (Ringdown) 階段，缺乏傳統長 Inspiral 特徵，導致傳統匹配濾波 (Matched Filtering) 效率下降。本項目旨在探討利用深度學習 (Deep Learning) 中的 ResNet 架構，是否能從高噪聲背景中自動識別此類瞬變訊號。

### 2. 數據增強策略 (Data Augmentation Strategy)
由於真實的 IMBH (中等質量黑洞) 觀測樣本極其稀缺，無法滿足深度學習的訓練需求。我們採用 **「模擬訊號注入 (Simulation Injection)」** 策略：
* **波形生成**：利用廣義相對論模型 `IMRPhenomXPHM` 生成模擬波形。
* **魔鬼訓練 (Hard Negative Mining)**：為了克服真實訊號微弱的問題，我們在訓練階段故意將信噪比 (SNR) 設定在極低的 **0.5 ~ 4.0** 區間，強迫模型學習在噪聲極大的環境下提取特徵。
* **噪聲抑制**：針對原始論文提到的環境干擾，我們實施了 **Notch Filter (陷波濾波器)**，強制濾除 60Hz 電源線及其倍頻干擾。

In [ ]:
# === Part A: 環境設置與工具函數 ===
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models
import numpy as np
import matplotlib.pyplot as plt
from pycbc.waveform import get_td_waveform
from scipy.signal import iirnotch, filtfilt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def apply_notch_filter(data, fs=2048):
    """移除 60Hz 電源線干擾及其倍頻 (雙重保險版)"""
    # 第一道防線：進來前先洗一次
    data = np.nan_to_num(data)

    for freq in [60, 120, 180]:
        b, a = iirnotch(freq, Q=30, fs=fs)
        # 執行濾波
        data = filtfilt(b, a, data)

    # 第二道防線 (關鍵！)：濾波完出去前，再洗一次！
    # 這能防止濾波過程中產生的新 NaN 污染後面的畫圖步驟
    data = np.nan_to_num(data, copy=False)

    return data

def to_spectrogram_image(strain, fs=2048):
    """將一維波形轉為標準化的二維頻譜圖 (Z-score Normalized)"""
    spec, _, _, _ = plt.specgram(strain, NFFT=128, Fs=fs, noverlap=110, visible=False)
    spec = np.log(spec + 1e-10)
    spec = np.nan_to_num(spec)
    if spec.std() > 0:
        spec = (spec - spec.mean()) / (spec.std() + 1e-8)
    else:
        spec = np.zeros_like(spec)
    return spec

# === Part B: 定義數據生成器 (Dataset) ===
class GWDataset(Dataset):
    def __init__(self, num_samples=1000, sample_rate=2048, duration=1.0):
        self.num_samples = num_samples
        self.data = []
        self.labels = []

        print(f"Generating {num_samples} training samples (Hard Mode: SNR 0.5-4.0)...")
        for i in range(num_samples):
            N = int(sample_rate * duration)
            noise = np.random.normal(0, 1, N)

            if i % 2 == 0:
                # 生成模擬黑洞波形 (IMRPhenomXPHM)
                m1 = np.random.uniform(50, 100)
                m2 = np.random.uniform(30, 80)
                hp, _ = get_td_waveform(approximant="IMRPhenomXPHM",
                                        mass1=m1, mass2=m2,
                                        delta_t=1.0/sample_rate,
                                        f_lower=20)
                hp.resize(N)

                # 低 SNR 設定，模擬真實微弱訊號
                snr_scale = np.random.uniform(0.5, 4.0)
                signal = hp.numpy() * 1e19 * snr_scale

                # 隨機平移與注入
                shift = np.random.randint(0, N - len(signal) if N>len(signal) else 1)
                combined = noise.copy()
                combined[shift:shift+len(signal)] += signal[:len(combined)-shift]

                # 濾波處理
                combined = apply_notch_filter(combined, sample_rate)
                self.data.append(combined)
                self.labels.append(1) # Label 1: Signal
            else:
                noise = apply_notch_filter(noise, sample_rate)
                self.data.append(noise)
                self.labels.append(0) # Label 0: Noise

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        ts = self.data[idx]
        spec = to_spectrogram_image(ts)
        # 轉為 PyTorch Tensor (Channel, H, W)
        spec_tensor = torch.from_numpy(spec).float().unsqueeze(0)
        label = torch.tensor(self.labels[idx]).float()
        return spec_tensor, label

# 準備數據集
train_dataset = GWDataset(num_samples=800)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

### 3. 模型架構與訓練 (Model Architecture & Training)
我們採用經典的 **ResNet-18** 架構，並針對重力波頻譜圖進行了以下修改：
1.  **輸入層調整**：將原本處理 RGB 圖像的 3 通道輸入改為 **單通道 (1-channel)**，以適配灰階頻譜圖。
2.  **輸出層調整**：將全連接層 (FC) 修改為單一神經元輸出，搭配 `BCEWithLogitsLoss` 進行二元分類。

訓練過程中，模型將學習區分「純噪聲」與「含有微弱黑洞訊號的噪聲」。

In [ ]:
# === Part C: 模型訓練 (含 Learning Curve 繪圖) ===

# 初始化模型
model = models.resnet18(weights=None)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(model.fc.in_features, 1)
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# === 新增：紀錄訓練過程的 list ===
loss_history = []
acc_history = []

print("\nStarting Training...")
epochs = 8
model.train()

for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).unsqueeze(1)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = (torch.sigmoid(outputs) > 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    # 計算平均 Loss 和 Acc
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total

    # 存起來畫圖用
    loss_history.append(epoch_loss)
    acc_history.append(epoch_acc)

    print(f"Epoch {epoch+1}/{epochs} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.2%}")

# === 訓練完成後，畫出學習曲線 ===
plt.figure(figsize=(12, 5))

# 左圖：Loss 曲線
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs+1), loss_history, 'b-o', label='Training Loss')
plt.title("Learning Curve: Loss")
plt.xlabel("Epoch")
plt.ylabel("BCE Loss")
plt.grid(True)
plt.legend()

# 右圖：Accuracy 曲線
plt.subplot(1, 2, 2)
plt.plot(range(1, epochs+1), acc_history, 'g-o', label='Training Accuracy')
plt.title("Learning Curve: Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.ylim(0, 1.0) # 固定範圍 0~100%
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

### 4. 實戰測試：GW190521 真實數據驗證

訓練完成後，我們將模型應用於真實的 GW190521 觀測數據。
**處理流程：**
1.  從原始數據中截取合併時刻 (Trigger Time) 前後 0.5 秒的片段。
2.  施加與訓練時相同的 **Notch Filter** 以移除 60Hz 干擾。
3.  轉換為頻譜圖並輸入 ResNet 進行推論。

**結果解讀：**
由於真實環境的噪聲分佈 (LIGO Real Noise) 遠比模擬的高斯噪聲複雜 (Sim-to-Real Gap)，且 GW190521 本身信號極其微弱，模型的信心分數可能不會達到極高值 (如 0.99)。然而，任何顯著高於背景底噪 (通常 < 0.01) 的分數，都暗示了該時間點存在潛在的異常結構。

In [ ]:
# === Part D: 實戰推論 (Inference on Real Data) ===
print("\nRunning Inference on Real GW190521 Data...")
model.eval()

# 檢查數據是否存在
if 'whitened_data' not in globals():
    print("❌ Error: 請先執行前面的數據下載步驟！")
else:
    # 1. 鎖定訊號位置 (位於數據開始後的第 5 秒)
    trigger_idx = int(5.0 * 2048) # 2048 is sample_rate
    half_window = int(0.5 * 2048)

    # 從 GPU 取回數據
    real_strain = whitened_data[0, 1, trigger_idx-half_window : trigger_idx+half_window].cpu().numpy()

    # 2. 關鍵前處理：濾除 60Hz 並轉頻譜圖
    real_strain_clean = apply_notch_filter(real_strain, fs=2048)
    spec_real_img = to_spectrogram_image(real_strain_clean, fs=2048)

    # 3. 模型預測
    real_tensor = torch.from_numpy(spec_real_img).float().unsqueeze(0).unsqueeze(0).to(device)
    with torch.no_grad():
        prob = torch.sigmoid(model(real_tensor)).item()

    # 4. 結果視覺化
    plt.figure(figsize=(12, 5))

    # 左圖：AI 看到的輸入
    plt.subplot(1, 2, 1)
    plt.imshow(spec_real_img, origin='lower', aspect='auto', cmap='viridis')
    plt.title(f"ResNet Input (Spectrogram)\nConfidence: {prob:.4f}")
    plt.xlabel("Time bin")
    plt.ylabel("Freq bin")
    plt.colorbar()

    # 右圖：去噪效果展示
    plt.subplot(1, 2, 2)
    plt.plot(real_strain[:300], label='Original', color='gray', alpha=0.5)
    plt.plot(real_strain_clean[:300], label='Notch Filtered', color='red', alpha=0.8)
    plt.title("Waveform Denoising (Zoom-in)")
    plt.legend()
    plt.grid(alpha=0.3)

    plt.tight_layout()
    plt.show()

    print("-" * 40)
    print(f"🚀 ResNet Detection Probability: {prob:.4f}")
    print("-" * 40)